In [1]:
import morphs
import numpy as np
import pickle

/usr/local/anaconda/envs/morphs36/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/usr/local/anaconda/envs/morphs36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def rasterize(spike_times, num_bins, window_size=.01):
    raster = np.zeros(num_bins)
    for spike_time in spike_times:
        raster[int(spike_time / window_size)] += 1
    return raster

In [3]:
window_size = .01 # seconds
stim_length = .4
num_bins = int(stim_length / window_size)

pop_tensors = {}
labels = {}
for block_path in morphs.config.paths.blocks():
    print(block_path)
    pop_tensors[block_path] = {}
    labels[block_path] = {}
    spikes = morphs.load.ephys_data(block_path, collapse_endpoints=False)
    spikes = spikes[
        (spikes["stim_aligned_time"] >= 0) & (spikes["stim_aligned_time"] < stim_length)
    ]
    cluster_map = {cluster:i for i, cluster in enumerate(np.sort(spikes['cluster'].unique()))}
    num_clusters = len(cluster_map)
    for morph_dim, morph_dim_group in spikes.groupby("morph_dim"):
        trial_gb = morph_dim_group.groupby(["morph_pos", "stim_presentation"])
        num_trials = len(trial_gb)
        pop_tensor = np.zeros((num_clusters, num_bins, num_trials))
        pos_list = np.zeros(num_trials)
        for i, ((morph_pos, stim_presentation), trial_block) in enumerate(trial_gb):
            pos_list[i] = morph_pos
            for cluster, cluster_group in trial_block.groupby("cluster"):
                pop_tensor[cluster_map[cluster], :, i] = rasterize(
                    cluster_group["stim_aligned_time"].values,
                    num_bins,
                    window_size=window_size,
                )
        pop_tensors[block_path][morph_dim] = pop_tensor

/mnt/cube/mthielk/analysis/B1101/kwik/Pen02_Lft_AP2501_ML500__Site03_Z2222__B1101_cat_P02_S03_1
/mnt/cube/mthielk/analysis/B1101/kwik/Pen02_Lft_AP2501_ML500__Site04_Z2410__B1101_cat_P02_S04_3
/mnt/cube/mthielk/analysis/B1101/kwik/Pen01_Lft_AP2500_ML750__Site04_Z2300__B1101_cat_P01_S04_3
/mnt/cube/mthielk/analysis/B1101/kwik/Pen01_Lft_AP2500_ML750__Site02_Z1880__B1101_cat_P01_S02_1
/mnt/cube/mthielk/analysis/B1101/kwik/Pen01_Lft_AP2500_ML750__Site03_Z2000__B1101_cat_P01_S03_2
/mnt/cube/mthielk/analysis/B1101/kwik/Pen02_Lft_AP2501_ML500__Site05_Z3072__B1101_cat_P02_S05_4
/mnt/cube/mthielk/analysis/B1101/kwik/Pen02_Lft_AP2501_ML500__Site02_Z2050__B1101_cat_P02_S02_1
/mnt/cube/mthielk/analysis/B1218/kwik/Pen01_Lft_AP2500_ML160__Site06_Z3368__B1218_cat_P01_S06_5
/mnt/cube/mthielk/analysis/B1218/kwik/Pen01_Lft_AP2500_ML160__Site05_Z2950__B1218_cat_P01_S05_3
/mnt/cube/mthielk/analysis/B1218/kwik/Pen01_Lft_AP2500_ML160__Site08_Z4323__B1218_cat_P01_S08_7
/mnt/cube/mthielk/analysis/B1218/kwik/Pe

In [4]:
savepath = morphs.paths.PROCESSED_DIR / "TDA.pkl"
with open(savepath.as_posix(), "wb") as f:
    pickle.dump({"pop_tensors": pop_tensors, "labels": labels}, f)

In [5]:
print(savepath.as_posix())

/usr/local/home/mthielk/GitHub/morphs/data/processed/TDA.pkl
